In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

torch.cuda.empty_cache()

# Qwen2.5 모델과 토크나이저 로드
model_name = "../Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map='cuda:0')
                                             # torch_dtype=torch.bfloat16)
initial_memory = torch.cuda.memory_allocated()
print(f"초기 GPU 메모리 사용량: {initial_memory / 1024**2:.2f} MB")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


초기 GPU 메모리 사용량: 5895.55 MB


In [2]:
# 샘플 입력 텍스트
sample_text = "안녕하세요! Qwen2.5 모델의 구조를 분석해보겠습니다."

# 입력 토큰화
inputs = tokenizer(sample_text, return_tensors="pt")
inputs = {k:v.to('cuda:0') for k, v in inputs.items()}
input_ids = inputs["input_ids"]

print("=== 모델 기본 정보 ===")
print(f"모델 이름: {model_name}")
print(f"입력 토큰 수: {input_ids.shape[1]}")
print(f"모델 구조: {model}")

=== 모델 기본 정보 ===
모델 이름: ../Qwen2.5-1.5B-Instruct
입력 토큰 수: 21
모델 구조: Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
   

In [3]:
# 각 레이어의 출력을 저장하기 위한 훅
layer_outputs = []
layer_memory_usage = []

def hook_fn(module, input, output):
    current_memory = torch.cuda.memory_allocated()
    layer_memory_usage.append(current_memory)
    layer_outputs.append(output[0].detach())

# 각 레이어에 훅 등록
hooks = []
hooks.append(model.model.embed_tokens.register_forward_hook(hook_fn))
for layer in model.model.layers:
    hooks.append(layer.register_forward_hook(hook_fn))
print(hooks)

# 모델 실행
with torch.no_grad():
    outputs = model(input_ids)

# 훅 제거 (메모리 누수 방지지)
for hook in hooks:
    hook.remove()

[<torch.utils.hooks.RemovableHandle object at 0x7f161c2c18d0>, <torch.utils.hooks.RemovableHandle object at 0x7f16107cedd0>, <torch.utils.hooks.RemovableHandle object at 0x7f16319c75b0>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c87c0>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8880>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4ca1d0>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4ca170>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8160>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8340>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c82e0>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8280>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8220>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8100>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c80a0>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c8040>, <torch.utils.hooks.RemovableHandle object at 0x7f14da4c83a0>, <torch.

In [4]:
print("\n=== 레이어별 출력 및 메모리 사용량 분석 ===")
for i, (output, memory) in enumerate(zip(layer_outputs, layer_memory_usage)):
    if i == 0:
        print(f"\nEmbedding layer 분석:")
    else:
        print(f"\n레이어 {i} 분석:")
    print(f"출력 텐서 크기: {output.shape}")
    print(f"GPU 메모리 사용량: {memory / 1024**2:.2f} MB")
    if i > 0:
        memory_diff = (memory - layer_memory_usage[i-1]) / 1024**2
        print(f"이전 레이어 대비 메모리 증가량: {memory_diff:.2f} MB")



=== 레이어별 출력 및 메모리 사용량 분석 ===

Embedding layer 분석:
출력 텐서 크기: torch.Size([21, 1536])
GPU 메모리 사용량: 5895.68 MB

레이어 1 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.11 MB
이전 레이어 대비 메모리 증가량: 8.44 MB

레이어 2 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.28 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 3 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.44 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 4 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.60 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 5 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.77 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 6 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5904.93 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 7 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5905.10 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 8 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5905.26 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 9 분석:
출력 텐서 크기: torch.Size([1, 21, 1536])
GPU 메모리 사용량: 5905.42 MB
이전 레이어 대비 메모리 증가량: 0.16 MB

레이어 10 분석

In [5]:

# 최종 출력 분석
print("\n=== 최종 출력 분석 ===")
logits = outputs.logits
print(f"로짓 텐서 크기: {logits.shape}")
print(f"다음 토큰 예측 확률: {torch.softmax(logits[0, -1], dim=-1).max().item():.4f}")

# 다음 토큰 예측
next_token = torch.argmax(logits[0, -1])
print(f"예측된 다음 토큰: {tokenizer.decode(next_token)}")


=== 최종 출력 분석 ===
로짓 텐서 크기: torch.Size([1, 21, 151936])
다음 토큰 예측 확률: 0.1686
예측된 다음 토큰:  Q


# logit을 이용하여 문장을 생성해보세요

In [6]:
# 샘플 입력 텍스트
sample_text = "안녕하세요! Qwen2.5 모델의 구조를 분석해보겠습니다."
current_memory = torch.cuda.memory_allocated()
"""
문장을 생성하는 코드 작성
"""

메모리 사용량: 121.42 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.12 MB
메모리 사용량: 6.56 MB
메모리 사용량: 6.51 MB
메모리 사용량: 5.59 MB
메모리 사용량: 14.45 MB
메모리 사용량: -2.13 MB
메모리 사용량: 7.96 MB
메모리 사용량: 5.00 MB
메모리 사용량: 16.95 MB


In [7]:
memory_diff = (memory - initial_memory) / 1024**2
print(f"총 메모리 사용량: {memory_diff:.2f} MB")
print(f"{sample_text}")


총 메모리 사용량: 244.18 MB
안녕하세요! Qwen2.5 모델의 구조를 분석해보겠습니다. Qwen2.5 모델은 1000개의 �
